In [125]:
from lxml import etree
import re
from operator import itemgetter


In [126]:
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from gensim import models
from gensim import similarities
from smart_open import smart_open

In [127]:
def extracttext(text):
    
    sentenses = re.split(r"[\n|\r]", text)
    # Remove leading and trailing spaces from each sentence
    results = []
    for sen in sentenses:
        s = sen.strip()
        if len(s):
            results.append(s)
            num_libros=len(results)
    #print("\n")
    return results[1]


In [128]:
def querysearch():
    
    queryid = []
    querytext = []
    a = 1
    queryall = [queryid ,querytext]
    while a!=47:
        if a < 10:
            b = "0"+str(a)
            if a == 4:
                a+=1
        elif a < 50:
                b = str(a)
                if a == 10 or a == 14 or a == 32 or a == 34 or a ==38 or a == 42:
                    a+=1
                elif a == 19 or a == 29:
                    a+=2

        docquer = etree.parse("./queries-raw-texts/wes2015.q"+b+".naf")
        raizquer=docquer.getroot()
        idquer = raizquer[0][1].get('publicId')
        textquery = raizquer[1].text
        queryid.append(idquer)
        querytext.append(textquery)
        a+=1
    #print(queryid)
    #print(querytext)    
    return queryall
    #print("\n")

In [129]:
#Gensim trae sus propias funciones para el procesamiento de texto
p = PorterStemmer()

<h2>Corpus Streaming</h2>
Normalmente los corpus reciden completamente en la memoria. Supongamos que hay millones de documentos en el corpus. Almacenarlos todos en RAM no será suficiente. En su lugar, supongamos que los documentos se almacenan en un archivo en el disco, un documento por línea. Gensim puede procesar un documento a la vez y actualizar dinamicamente e calculo de IDF y del vocabulario.

In [130]:
#Procesamiento de un texto utilizando las funciones de Gensim
def process(text):
    doc_nor = text.lower()
    #doc_reg = re.sub(r'[^\w|^\d]', ' ', doc_nor)
    #doc_reg = re.sub(r'[^a-zA-Z]', ' ', doc_nor)
    doc_sw = remove_stopwords(doc_nor)
    doc_stem = PorterStemmer().stem_sentence(doc_sw)
    return doc_stem.split()

In [131]:
#Creaccion de lista con documentos procesados
docDict = []
docTitle = []
docID = []
a = 1

while a!=332:
    if a < 10:
        b = "00"+str(a)
    elif a < 100:
            b = "0"+str(a)
    else:
            b = str(a)
    doc = etree.parse("./docs-raw-texts/wes2015.d"+b+".naf")
    raiz=doc.getroot()
    titulo = raiz[0][0].get('title')
    docTitle.append(titulo)
    iddoc = raiz[0][1].get('publicId')
    docID.append(iddoc)
    #print(iddoc)
    #print(titulo)
    text = raiz[1].text
    a+=1
    line = extracttext(text)
    #print("\n")
    docDict.append(process(line))

#print(docID[2])
#print(docTitle[2])
#print(docDict[2]) #Documentos ya procesados, de esta forma deben ser enviados al metodo de Gensim



In [132]:
#Ahora si llamamos al metodo de Gensim para crear el diccionario a partir de los documentos
dictionary = corpora.Dictionary(docDict)
dictionary.save('clase3.dict')
#print(dictionary)

#Notese que Gensim de una vez asigna a cada token un id
#print(dictionary.token2id)

In [133]:
new_doc = "Human media interaction media"
new_doc_bow = dictionary.doc2bow(process(new_doc))
print(new_doc_bow)  # ¿Si hay cuatro palabras pq solo dos tokens?

[(70, 1), (4236, 1), (7809, 2)]


In [164]:
#Construccion del corpus == Cada documento representado con el vocabulario/diccionario definido anteriormente (Bolsa de palabras)
class MyCorpus(object):
    def __iter__(self):
        a = 1

        while a!=332:
            if a < 10:
                b = "00"+str(a)
            elif a < 100:
                    b = "0"+str(a)
            else:
                b = str(a)
            doc = etree.parse("./docs-raw-texts/wes2015.d"+b+".naf")
            raiz=doc.getroot()
            titulo = raiz[0][0].get('title')
            iddoc = raiz[0][1].get('publicId')
            #print(iddoc)
            #print(titulo)
            text = raiz[1].text
            a+=1
            line = extracttext(text)
            yield dictionary.doc2bow(process(line))

corpus_memory_friendly = MyCorpus()#Todos mis documentos ahora estan representados como una bolsa de palabras
#Almaceno mi corpus
corpora.MmCorpus.serialize('corpus.mm', corpus_memory_friendly)  

In [165]:
#Cargo mi corpus
corpus = corpora.MmCorpus('corpus.mm')
#print(corpus) #No lo carga en memoria
#Para leer la representacion de bolda de palabras resultante de cada documento en el corpus
#for doc in corpus:
#    print(doc)

<h2>Vector Space Model - TF-IDF</h2>

In [166]:
#Suponga que tenemos el diccionario y corpus en disco
#Primero necesitamos cargarlos
dictionary = corpora.Dictionary.load('clase3.dict')
corpus = corpora.MmCorpus('corpus.mm')

In [167]:
#Construyendo un modelo VSM con ponderacion por TF-IDF
tfidf = models.TfidfModel(corpus) #Inizializacion

In [168]:
#Supongamos que queremos calcular la similitud de un nuevo documento con los documentos en el corpus
#1. Construimos la representación vectorial del query
query_doc = "Amazon vs Microsoft"
query_doc_bow = dictionary.doc2bow(process(query_doc))
print(query_doc_bow)
print(tfidf[query_doc_bow])

[(16449, 1)]
[(16449, 1.0)]


In [169]:
#2. Ahora construimos la matrix t/d con los documentos contra los cuales queremos compara el query
index = similarities.MatrixSimilarity(tfidf[corpus]) 
# Vamos a salvar el inidice resultante para no tener que recalcularlo cada vez que los necesitemos.
index.save('clase3tfidf.index')

In [170]:
#Una vez guardado lo podemos cargar sin necesidad de recalcularlo
index = similarities.MatrixSimilarity.load('clase3tfidf.index')

In [171]:
# Ejecutar las similitudes entre el documento query y el corpus ahora es muy facil!!!

queryreturn = querysearch()
queryid = queryreturn[0]
querydoc = queryreturn[1]
archivo = open ('top-10.txt','w')
id=0
for query in querydoc:
    
    query_doc_bow = dictionary.doc2bow(process(query))
    sims = index[tfidf[query_doc_bow]]
    #print(type(sims))
    listofdoctf= list(enumerate(sims))
    #print(listofdoctf)
    #print("\n")
    sorted_by_second = sorted(listofdoctf,key=itemgetter(1), reverse = True)
    #print(sorted_by_second)
    varTen = "10 most similarity documents:\n"
    #archivo.write(varTen)
    archivo.write(queryid[id]+"\t")
    for doc, similitudQ in sorted_by_second[:20]:  
        
        archivo.write( docID[doc]+",")
        #print("{}\t{}".format(docID[doc], similitudQ))
    #print("\n")    
    archivo.write("\n")
    id+=1
archivo.close()
    

#print(list(enumerate(sims)))  print (document_number, document_similarity) 2-tuples#

In [185]:

def relevancejud(docDic):
    dicFinal = []
    
    
    #print(len(docDic))
    
    for index in range(len(docDic)):
        dataquery = docDic[index][1]
        #print(dataqueryid)
        #print("\n")
        #print(dataquery)
        #print("\n")
        normalice = re.sub(r':\d', '', dataquery)
        normalice = re.split(",",normalice)
        #print(normalice)        
        dicFinal.append(normalice)

        #print(dicFinal)
        #print(dicFinal2)
    return dicFinal


In [186]:

def docssimulationDict(docDic):
    dicFinal2 = []
    
    #print(len(docDic))
    
    for index in range(len(docDic)):
        dataquery = docDic[index][1]
        #print(dataqueryid)
        #print("\n")
        #print(dataquery)
        #print("\n")
        normalice = re.sub(r':\d', '', dataquery)
        normalice = re.split(",",normalice)
        #print(normalice)
        dicFinal2.append(normalice)

        #print(dicFinal3)
        #print(dicFinal4)
    return dicFinal2


In [187]:
docsrelevantes=[]

def precisionandrecall(relevancequery,docsimquery):
    
    for rel in relevancequery:
        for rel1 int range(len(rel)):
            if rel[rel1] in docsimquery:
                print("hola")
    
    
    return docsrelevantes
    

In [199]:
docDict1 = []
for line in smart_open('top-10.txt', 'rb'):
    docDict1.append(process(line))  

docDict2 = []
for line in smart_open('relevance-judgments.tsv', 'rb'):
    docDict2.append(process(line))  
#print(docDict2)
dicdocsim = docssimulationDict(docDict1)
dicjud = relevancejud(docDict2)


#docsrell = precisionandrecall(dicjud2,dicdocsim2,3)
print(dicdocsim[0])
print("\n")
print(dicjud[0][1])
print(len(dicjud[3]))

#print("\n")

['d016', 'd259', 'd209', 'd254', 'd085', 'd186', 'd170', 'd008', 'd315', 'd004', 'd154', 'd163', 'd060', 'd089', 'd296', 'd100', 'd094', 'd145', 'd059', 'd179', '']


d254
7
